In [3]:
%%bash 

wget 'ftp://ftp.ebi.ac.uk/pub/databases/intact/various/ot_graphdb/2021-02-24/data/interactor_pair_interactions.json' \
    -O /Users/dsuveges/project_data/interactions/interactor_pair_interactions_2021-02-24.json
    
ls -la /Users/dsuveges/project_data/interactions

total 1377744
drwxrwxr-x  5 dsuveges  384566875        160 26 Feb 11:11 .
drwxrwxr-x  7 dsuveges  384566875        224 10 Feb 17:32 ..
drwxrwxr-x  3 dsuveges  384566875         96 21 Jan 15:45 2021.02.21_interactions
-rw-rw-r--  1 dsuveges  384566875  693807138 26 Feb 11:16 interactor_pair_interactions_2021-02-24.json
drwxrwxr-x  3 dsuveges  384566875         96 10 Nov 17:50 unmapped


--2021-02-26 11:11:56--  ftp://ftp.ebi.ac.uk/pub/databases/intact/various/ot_graphdb/2021-02-24/data/interactor_pair_interactions.json
           => ‘/Users/dsuveges/project_data/interactions/interactor_pair_interactions_2021-02-24.json’
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/intact/various/ot_graphdb/2021-02-24/data ... done.
==> SIZE interactor_pair_interactions.json ... 693807138
==> PASV ... done.    ==> RETR interactor_pair_interactions.json ... done.
Length: 693807138 (662M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  652K 17m19s
    50K .......... .......... .......... .......... ..........  0% 2.16M 11m13s
   100K .......... .......... .......... .......... ..........  0% 2.85M 8m46s
   150K .......... .......... ....

In [4]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd

global spark
spark = (
    pyspark.sql
     .SparkSession
     .builder
     .config("spark.driver.memory", "10g")
     .getOrCreate()
    )
print('Spark version: ', spark.version)

# Reading data:
intact_data = '/Users/dsuveges/project_data/interactions/interactor_pair_interactions_2021-02-24.json'
df = spark.read.json(intact_data)
df.show()

Spark version:  3.0.0
+--------------------+--------------------+--------------------+-------------+
|         interaction|         interactorA|         interactorB|  source_info|
+--------------------+--------------------+--------------------+-------------+
|[, [[,, Homo sapi...|[unspecified role...|[unspecified role...|[235, intact]|
|[, [[,, Homo sapi...|[unspecified role...|[unspecified role...|[235, intact]|
|[, [[,, Homo sapi...|[unspecified role...|[unspecified role...|[235, intact]|
|[, [[,, Homo sapi...|[unspecified role...|[unspecified role...|[235, intact]|
|[, [[,, Homo sapi...|[regulator, URS00...|[regulator target...|[235, intact]|
|[, [[,, Homo sapi...|[regulator target...|[regulator, URS00...|[235, intact]|
|[, [[,, Homo sapi...|[regulator, URS00...|[regulator target...|[235, intact]|
|[, [[,, Homo sapi...|[unspecified role...|[unspecified role...|[235, intact]|
|[, [[MI:1060, spo...|[unspecified role...|[unspecified role...|[235, intact]|
|[, [[,, Homo sapi...|[regulat

```
{
  "interactorA": {
    "organism": {
      "taxon_id": 9606,
      "mnemonic": "human",
      "scientific_name": "Homo sapiens"
    },
    "id_source": "rnacentral",
    "id": "URS0000574A2C_9606",
    "biological_role": "unspecified role"
  },
  "interactorB": {
    "organism": {
      "taxon_id": 9606,
      "mnemonic": "human",
      "scientific_name": "Homo sapiens"
    },
    "id_source": "rnacentral",
    "id": "URS000075C808_9606",
    "biological_role": "unspecified role"
  },
  "source_info": {
    "database_version": "235",
    "source_database": "intact"
  },
  "interaction": {
    "causal_interaction": null,
    "evidence": [
      {
        "pubmed_id": "29473328",
        "participant_detection_method_B": [
          {
            "mi_identifier": "MI:1196",
            "short_name": "qrt-pcr"
          }
        ],
        "interaction_detection_method_mi_identifier": "MI:0096",
        "interaction_detection_method_short_name": "pull down",
        "participant_detection_method_A": [
          {
            "mi_identifier": "MI:1196",
            "short_name": "qrt-pcr"
          },
          {
            "mi_identifier": "MI:0396",
            "short_name": "predetermined"
          }
        ],
        "expansion_method_mi_identifier": null,
        "host_organism_tax_id": 9606,
        "interaction_type_mi_identifier": "MI:0915",
        "expansion_method_short_name": null,
        "host_organism_scientific_name": "Homo sapiens Caucasian breast adenocarcinoma",
        "interaction_identifier": "EBI-25566870",
        "interaction_type_short_name": "physical association"
      }
    ],
    "interaction_score": 0.4
  }
}
```

In [5]:
get_id = udf(       
    lambda arr: [x['interaction_identifier'] for x in arr],
    ArrayType(elementType=StringType())
)
summary_df = (
    df
    .select(
        col('interactorA.id').alias('interactorA'),
        col('interactorA.biological_role').alias('biological_role_A'),
        col('interactorB.id').alias('interactorB'),
        col('interactorB.biological_role').alias('biological_role_B'),
        col('interaction.evidence').alias('evidence'),
        col('source_info.source_database').alias('source')
    )
    .withColumn('ids', get_id(col('evidence')))
    .drop('evidence')
    .toPandas()
)

summary_df.head()

,interactorA,biological_role_A,interactorB,biological_role_B,source,ids
0,URS0000574A2C_9606,unspecified role,URS000075C808_9606,unspecified role,intact,[EBI-25566870]
1,Q9UKV8,unspecified role,URS0000811F6D_9606,unspecified role,intact,[EBI-25466487]
2,URS0000192F9C_9606,unspecified role,URS0000811F6D_9606,unspecified role,intact,[EBI-25466475]
3,URS0000811F6D_9606,unspecified role,URS000014D9C1_9606,unspecified role,intact,[EBI-25466466]
4,URS0000811F6D_9606,regulator,URS000008DA94_9606,regulator target,intact,[EBI-25466446]


In [8]:
int_a_counts = summary_df.interactorA.value_counts()
int_b_counts = summary_df.interactorB.value_counts()

In [56]:
filters = ['signor', 'reactome', 'intact']


for f in filters:
    print(f'\nApplied filter: {f}')
    int_a_counts = summary_df.loc[summary_df.source == f].interactorA.value_counts()
    int_b_counts = summary_df.loc[summary_df.source == f].interactorB.value_counts()
    print(
        pd.DataFrame(int_b_counts)
        .join(pd.DataFrame(int_a_counts))
        .fillna(0)
        .assign(summary = lambda x: x['interactorB'] +  x['interactorA'])
        .sort_values(by=['summary'], ascending=False)
        .head(10)
        .assign(symbol = lambda x: [get_symbol(y) for y in x.index])
    )


Applied filter: signor
        interactorB  interactorA  summary   symbol
P28482           18        167.0    185.0    MAPK1
P17612           39        144.0    183.0   PRKACA
P31749           27        151.0    178.0     AKT1
P27361           48        117.0    165.0    MAPK3
P12931           88         71.0    159.0      SRC
P06493           29        113.0    142.0     CDK1
P68400           27        111.0    138.0  CSNK2A1
P04637            7        126.0    133.0     TP53
Q16539           27        104.0    131.0   MAPK14
P63096            6        120.0    126.0    GNAI1

Applied filter: reactome
        interactorB  interactorA  summary  symbol
P62993           46        133.0    179.0    GRB2
P12931          106         68.0    174.0     SRC
P62805            3        141.0    144.0    H4C1
Q07889           48         96.0    144.0    SOS1
P27986            3        134.0    137.0  PIK3R1
P42336           65         66.0    131.0  PIK3CA
P63165           31         98.0    129

In [30]:
import requests


def get_symbol(uid):
    r = requests.get(f'http://rest.ensembl.org/xrefs/name/human/{uid}?content-type=application/json')
    return [x['display_id'] for x in r.json() if x['dbname'] == 'Uniprot_gn'][0]



In [47]:
summary_df.source.value_counts()

intact      377378
reactome     22557
signor       12241
Name: source, dtype: int64

In [55]:
f  = 'reactome'
summary_df.loc[summary_df.source == f].interactorB.value_counts()

P12931    106
P0DP23     91
P01906     81
P20036     74
P09693     72
         ... 
Q16363      1
P50053      1
Q5VWX1      1
P55064      1
Q9Y2T1      1
Name: interactorB, Length: 4769, dtype: int64